In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../')
import velovae as vv

In [2]:
dataset = 'IPSC'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')

In [ ]:
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20, compute_umap=True)
#adata.write_h5ad(f'data/{dataset}_pp.h5ad')

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/continuous/{dataset}'
gene_plot = ['Vim','Nr2f1', 'Krt7', 'H19']

# Informative Time Prior

In [4]:
tprior = adata.obs["day"].to_numpy()
tprior = np.array([float(x) for x in tprior])
adata.obs["tprior"] = tprior - tprior.min()

# Train a Vanilla VAE

In [5]:
figure_path = f'{figure_path_base}/Vanilla'
model_path = f'{model_path_base}/Vanilla'

torch.manual_seed(2022)
np.random.seed(2022)

vanilla_vae = vv.VanillaVAE(adata, 
                            tmax=20, 
                            device='cuda:0',
                            init_method='tprior',
                            init_key='tprior',
                            tprior='tprior')

vanilla_vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path, embed='embed')

vanilla_vae.save_model(model_path, 'encoder', 'decoder')
vanilla_vae.save_anndata(adata, 'vanilla', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
------------------------- Train a Vanilla VAE -------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 146, test iteration: 290
********* Early Stop Triggered at epoch 181. *********
*********              Finished. Total Time =   0 h :  5 m :  2 s             *********
Final: Train ELBO = 82.072,           Test ELBO = 81.929


# Train a VeloVAE

In [6]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             init_method='tprior',
             init_key='tprior',
             tprior='tprior',
             init_ton_zero=False)

vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path, embed='embed')

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 146, test iteration: 290
*********       Stage 1: Early Stop Triggered at epoch 390.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/26682 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.017
Average Set Size: 773
Finished. Actual Time:   0 h :  0 m : 31 s
*********       Stage 2: Early Stop Triggered at epoch 547.       *********
*********              Finished. Total Time =   0 h : 17 m : 21 s             *********
Final: Train ELBO = 1373.160,           Test ELBO = 1375.326


# Train a FullVB

In [7]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAEFullVB(adata, 
                       tmax=20, 
                       dim_z=5, 
                       device='cuda:0',
                       init_method='tprior',
                       init_key='tprior',
                       tprior='tprior',
                       init_ton_zero=False)

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path, embed='embed')

full_vb.save_model(model_path, 'encoder','decoder')
full_vb.save_anndata(adata, 'fullvb', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 146, test iteration: 290
*********       Stage 1: Early Stop Triggered at epoch 221.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/26682 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.011
Average Set Size: 836
Finished. Actual Time:   0 h :  0 m : 33 s
*********       Stage 2: Early Stop Triggered at epoch 275.       *********
*********              Finished. Total Time =   0 h : 10 m : 41 s             *********
Final: Train ELBO = 1337.518,           Test ELBO = 1334.156


# Train a Branching ODE

In [8]:
figure_path = f'{figure_path_base}/BrODE'
model_path = f'{model_path_base}/BrODE'

torch.manual_seed(2022)
np.random.seed(2022)
brode = vv.BrODE(adata, 'clusters', 'fullvb_time', 'fullvb_z')

brode.train(adata, 'fullvb_time', 'clusters', plot=False, gene_plot=gene_plot, figure_path=figure_path)

brode.save_model(model_path, 'brode')
brode.save_anndata(adata, 'brode', data_path, file_name=f'{dataset}.h5ad')

vv.plot_transition_graph(adata, save=f'{figure_path_base}/transition.png')

Graph Partition
Number of partitions:  2
Computing type-to-type transition probability
Obtaining the MST in each partition
Initialization using type-specific dynamical model.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

------------------------ Train a Branching ODE ------------------------
             Epithelial  IPS  MET  Neural  Stromal  Trophoblast
Epithelial          0.0  0.0  1.0     0.0      0.0          0.0
IPS                 0.0  0.0  1.0     0.0      0.0          0.0
MET                 0.0  0.0  1.0     0.0      0.0          0.0
Neural              0.0  0.0  1.0     0.0      0.0          0.0
Stromal             1.0  0.0  0.0     0.0      0.0          0.0
Trophoblast         1.0  0.0  0.0     0.0      0.0          0.0
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 146, test iteration: 290
*********           Early Stop Triggered at epoch 112.   

# Evaluation

In [ ]:
cluster_edges = [('MET','Epithelial'),
                 ('Epithelial','IPS'),
                 ('Epithelial','Neural'),
                 ('Epithelial','Trophoblast'),
                 ('Epithelial','Stromal')]
vv.post_analysis(adata,
                 'eval',
                 ['Vanilla VAE','VeloVAE','FullVB','BrODE'],
                 ['vanilla','velovae','fullvb','brode'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=figure_path_base,
                 cluster_edges=cluster_edges)